In [1]:
from math import log10, sqrt
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim_frame
import imageio
import matplotlib.pyplot as plt
from PIL import Image
import cpbd
from deepface import DeepFace as DF
import pandas as pd

def check(true, pred):
    # check for format, it needs to be imageio.plugins.ffmpeg.FfmpegFormat.Reader"
    assert type(true) == imageio.plugins.ffmpeg.FfmpegFormat.Reader
    
    # check for frame counts, they need to be equal
    l1 = true.count_frames()
    l2 = pred.count_frames()
    print(l1)
    print(l2)
    #assert l1 == l2 # "Different frame numbers!"
    
    # check for different image resolutions
    assert np.array(true.get_data(0)).shape == np.array(pred.get_data(0)).shape
    
def check_v2(pred):
    # check for format, it needs to be imageio.plugins.ffmpeg.FfmpegFormat.Reader"
    assert type(pred) == imageio.plugins.ffmpeg.FfmpegFormat.Reader
    
def to_grayscale(im):
    return np.array(im[:, :, 0] * 0.2989 + im[:, :, 1] * 0.5870 + im[:, :, 2] * 0.1140)

# metric 1: PSNR, Peak Signal to Noise Ratio
# range: usuall 25-50dB, higher the score, better the prediction
def eval_psnr(true, pred, offset = 0):
    #check(true, pred)
    def psnr_frame(true, pred):
        mse = np.mean((true - pred) ** 2)
        if (mse == 0):
            return 100
        return 20 * log10(np.max(true) / sqrt(mse))
    
    psnrs = []
    l1 = true.count_frames()
    for i in range(30):
        psnrs.append(psnr_frame(true.get_data(i), pred.get_data(i)))
    return sum(psnrs) / 30

# metric 2: SSIM, Structural Similarity Index
# range: 0-1, higher the score, similar the images
def eval_ssim(true, pred, offset = 0):
    #check(true, pred)
    
    ssims = []
    l1 = true.count_frames()
    for i in range(30):
        ssims.append(ssim_frame(true.get_data(i), pred.get_data(i), channel_axis=2))
    return sum(ssims) / 30

# metric 3: CPBD, Cumulative Probability Blur Detection
# range: 0-1, higher the score, sharper (less blur) the image
def eval_cpbd(pred):
    check_v2(pred)
    
    cpbds = []
    l1 = pred.count_frames()
    for i in range(l1):
        cpbds.append(cpbd.compute(to_grayscale(pred.get_data(i))))
    return sum(cpbds) / l1

# metric 4.1: ACD-I, Average Content Distance-Identity
def eval_acdi(pred):
    check_v2(pred)
    
    acdis = []
    l1 = pred.count_frames()
    prev_grayscale = to_grayscale(pred.get_data(0))
    for i in range(1, l1):
        current_grayscale = to_grayscale(pred.get_data(i))
        acdis.append( np.mean((current_grayscale - prev_grayscale)**2) )
        prev_grayscale = current_grayscale
    return sum(acdis) / (l1 - 1)

# metric 4.2: ACD-C, Average Content Distance-Context
def eval_acdc(pred):
    check_v2(pred)
    filename = "im3.jpg"
    
    acdcs = []
    l1 = pred.count_frames()
    Image.fromarray(pred.get_data(0)).save(filename)
    prev_rep = np.array(DF.represent(filename, enforce_detection = False, model_name = "OpenFace"))
    for i in range(1, l1):
        Image.fromarray(pred.get_data(i)).save(filename)
        current_rep = DF.represent(filename, enforce_detection = False, model_name = "OpenFace")
        acdcs.append( np.mean((np.array(current_rep[0]['embedding']) - np.array(prev_rep[0]['embedding']))**2) )
        prev_rep = current_rep
    return sum(acdcs) / (l1 - 1)
    

In [2]:
original_path = "results_comparison_resized/"


64 64


In [ ]:
for i in range(1,15):
    model_path = f"results_exp{i}/"
    import os, time
    original_files = os.listdir(original_path)
    model_files = os.listdir(model_path)
    print(len(model_files),len(original_files))

    s = time.time()
    model_evals = {}
    count = 0
    psnr_sum = 0
    ssim_sum = 0
    cpbd_sum = 0
    acdi_sum = 0
    acdc_sum = 0
    for video_name in original_files:
        try:
            orig_vid = imageio.get_reader(original_path + "/" + video_name, 'ffmpeg')
            model_vid = imageio.get_reader(model_path + "/" + video_name, 'ffmpeg')
        except:
            print(f"missing either {sda_video_name} or {mit_video_name} or {sda_resized_video_name} skipping!")
            missing_files.append(video_name)
            continue
        count+=1
        # 1. eval psnr
        psnr_model = eval_psnr(orig_vid, model_vid, offset=1)
        psnr_sum += psnr_model
        # 2. eval ssim
        ssim_model = eval_ssim(orig_vid, model_vid, offset=1)
        ssim_sum += ssim_model
        # 3. eval cpbd
        cpbd_model = eval_cpbd(model_vid)
        cpbd_sum += cpbd_model
        # 4.1. eval acdi
        acdi_model = eval_acdi(model_vid)
        acdi_sum += acdi_model
        # 4.2. eval acdc
        acdc_model = eval_acdc(model_vid)
        acdc_sum += acdc_model
        
    print(f"Experiment {i} PSNR: {psnr_sum/count} SSIM: {ssim_sum/count} CPBD: {cpbd_sum/count} ACDI: {acdi_sum/count} ACDC: {acdc_sum/count}")

64 64
Experiment 1 PSNR: 27.881303233166136 SSIM: 0.18973646889282308 CPBD: 0.8111896591123989 ACDI: 50.72264043414669 ACDC: 0.00023048413600882845
64 64
Experiment 2 PSNR: 27.94531101866401 SSIM: 0.3548698106940581 CPBD: 0.7085449599674895 ACDI: 48.88564578601913 ACDC: 0.00025871339879650796
64 64
Experiment 3 PSNR: 27.91337469487995 SSIM: 0.3900276199816639 CPBD: 0.6583706315193628 ACDI: 89.45407337659374 ACDC: 0.000275750139998541
64 64
Experiment 4 PSNR: 27.96943783166812 SSIM: 0.20537214478892843 CPBD: 0.8465898624371925 ACDI: 25.387568079936855 ACDC: 0.00019107355323825392
64 64
Experiment 5 PSNR: 29.555944074346797 SSIM: 0.5885742620918432 CPBD: 0.48381223390061 ACDI: 46.82682719096281 ACDC: 0.00018147509343378613
64 64
Experiment 6 PSNR: 28.03028475988021 SSIM: 0.4385125621604223 CPBD: 0.6223542420869923 ACDI: 25.834058368150494 ACDC: 0.0002063498588167557
64 64
Experiment 7 PSNR: 27.946370122490077 SSIM: 0.38232616372335043 CPBD: 0.6364217778059766 ACDI: 0.1731186900152249 ACD

{'video28.mp4': [28.20293022613264,
  0.25779083416504384,
  0.8145497983661941,
  48.85846154702031,
  0.00020220011440490906],
 'video45.mp4': [28.153944801751017,
  0.2519167384581882,
  0.7823900919464107,
  47.32517540505449,
  0.00012767127903655647],
 'video5.mp4': [28.17137504174448,
  0.15671148193507622,
  0.8183321328348714,
  55.49745162569092,
  0.000297791312592014],
 'video58.mp4': [28.592718006676055,
  0.28842922255705583,
  0.7936994292723166,
  39.61157163998157,
  0.00030567086045552765],
 'video53.mp4': [27.824058062962877,
  0.16026669840259014,
  0.8145037220897959,
  55.838541127710066,
  0.00025621356355145494],
 'video30.mp4': [28.35899127483152,
  0.21634308729671003,
  0.7904281433734144,
  38.421316032782144,
  0.00011964047747388221],
 'video17.mp4': [28.09212696952552,
  0.26231732495802623,
  0.809949532346813,
  48.45891324929626,
  0.0002885622705954892],
 'video27.mp4': [28.280294882872422,
  0.1995593326746963,
  0.7792252669467917,
  38.066208980500